In [54]:
import rospy
import tf
from kuka_arm.srv import *
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from geometry_msgs.msg import Pose
from mpmath import *
from sympy import *
import numpy as np

In [55]:
# Define DH param symbols
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7') # twist angle
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7') # joint length
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') # joint offset

# Joint angle symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # joint variables: theta_i
roll, pitch, yaw = symbols('roll pitch yaw')

In [56]:
def rot_x(angle):
    return Matrix([[ 1,         0,            0],
                   [ 0, cos(angle), -sin(angle)],
                   [ 0, sin(angle),  cos(angle)]])
def rot_y(angle):
    return Matrix([[ cos(angle),  0, sin(angle)],
                   [          0,  1,          0],
                   [-sin(angle),  0, cos(angle)]])
def rot_z(angle):
    return Matrix([[  cos(angle), -sin(angle), 0],
                   [  sin(angle),  cos(angle), 0],
                   [      0,        0,         1]])

def DH_matrix(alpha, a, d, q):
    return Matrix([[            cos(q),           -sin(q),           0,             a],
                   [ sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
                   [ sin(q)*sin(alpha), cos(q)*sin(alpha),  cos(alpha),  cos(alpha)*d],
                   [                 0,                 0,           0,             1]])

In [57]:
# Modified DH params
s = {alpha0:     0, a0:      0, d1:  0.75,
     alpha1: -pi/2, a1:   0.35, d2:     0, q2: q2-pi/2,
     alpha2:     0, a2:   1.25, d3:     0,
     alpha3: -pi/2, a3: -0.054, d4:  1.50,
     alpha4:  pi/2, a4:      0, d5:     0,
     alpha5: -pi/2, a5:      0, d6:     0,
     alpha6:     0, a6:      0, d7: 0.303, q7: 0}

# Define Modified DH Transformation matrix

T0_1 = DH_matrix(alpha0, a0, d1, q1).subs(s)        
T1_2 = DH_matrix(alpha1, a1, d2, q2).subs(s)        
T2_3 = DH_matrix(alpha2, a2, d3, q3).subs(s)

T3_4 = DH_matrix(alpha3, a3, d4, q4).subs(s)        
T4_5 = DH_matrix(alpha4, a4, d5, q5).subs(s)        
T5_6 = DH_matrix(alpha5, a5, d6, q6).subs(s)        
T6_G = DH_matrix(alpha6, a6, d7, q7).subs(s)

In [91]:
R_roll = rot_x(roll)
R_pitch = rot_y(pitch)
R_yaw = rot_z(yaw)

R_z = rot_z(pi)
R_y = rot_y(-pi/2)
R_corr = R_z * R_y

Rrpy = R_yaw * R_pitch * R_roll * R_corr

T0_3 = T0_1 * T1_2 * T2_3
#        T0_G = T0_3 * T3_4 * T4_5 * T5_6 * T6_G
#        T_total = T0_G * R_corr

R0_3 = T0_3[0:3, 0:3]

R3_6 = R0_3**(-1) * Rrpy

In [59]:
T0_G = T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_G
# T0_G*R_corr

In [78]:
subs_theta = {q1: -0.95, q2: -0.19, q3: 0.08, q4: 6.06, q5:2.15, q6: 1.74}

In [79]:

(T0_G*R_corr).evalf(subs=subs_theta)

Matrix([
[-0.415023115644073,  0.277457922339151, -0.866471531449434, 0.811216459954351],
[ 0.261911572623822, -0.875613770385484, -0.405835993028013, -1.23088118025654],
[-0.871296816020122, -0.395370239702685,  0.290730514309452,  1.82449618060859],
[                 0,                  0,                  0,               1.0]])

rviz shows 0.81808, -1.2302, 1.827

In [80]:
# from 0,0,0 of G to the base_link

(T0_G*R_corr).evalf(subs=subs_theta)*Matrix([0, 0, 0, 1])

Matrix([
[0.811216459954351],
[-1.23088118025654],
[ 1.82449618060859],
[              1.0]])

In [83]:
Matrix([0,0,1]).shape

(3, 1)

In [86]:
R3_6 = simplify(T3_4 * T4_5 * T5_6)[0:3, 0:3]
R3_6

Matrix([
[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4)],
[                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5)],
[-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5)]])

In [84]:
r12, r13 = R[0,1], R[0,2]
r21, r22, r23 = R[1,0], R[1,1], R[1,2] 
r32, r33 = R[2,1], R[2,2]
# Euler angles from rotation matrix
q5 = atan2(sqrt(r13**2 + r33**2), r23)
q4 = atan2(r33, -r13)
q6 = atan2(-r22, r21)

3.141592653589793

## Test IK

## Validation


In [124]:
position = [1.8476,1.1,1.0194]
orientation = [0.744158,0.474277,-0.209642,0.421118]

px, py, pz = position[0], position[1], position[2]
ee_roll, ee_pitch, ee_yaw = tf.transformations.euler_from_quaternion(
    [orientation[0], orientation[1],orientation[2], orientation[3]])

In [125]:
R_roll = rot_x(roll)
R_pitch = rot_y(pitch)
R_yaw = rot_z(yaw)

R_z = rot_z(pi)
R_y = rot_y(-pi/2)
R_corr = R_z * R_y

Rrpy = R_yaw * R_pitch * R_roll * R_corr

In [151]:
P_EE = Matrix([px, py, pz])
P_WC = P_EE - Rrpy.evalf(subs={roll: ee_roll, pitch: ee_pitch, yaw: ee_yaw}) * Matrix([0, 0, s[d7]])

P_WC_x, P_WC_y, P_WC_z = P_WC[0], P_WC[1], P_WC[2]

# 2. calculate theta1:
theta1 = atan2(P_WC_y, P_WC_x).evalf()

# 3. calculate theta2:
# S1: distance between the x-y-plane projections of 2 and 5
# s2: P_WC_Z - d1
# s3: distance between 2, 5
# s4: distance between 3, 5
# beta1: atan2(s2, s1)
# beta2: angle_325
# beta3: angle_235
# beta4: pi/2 - angle_23(5^o)

S1 = sqrt(P_WC_x**2 + P_WC_y**2) - s[a1]
S2 = P_WC_z - s[d1]
S3 = sqrt(S1**2 + S2**2)
S4 = sqrt(s[d4]**2 + s[a3]**2)
beta1 = atan2(S2, S1)
cos_beta2 = (s[a2]**2 + S3**2 - S4**2)/(2*s[a2]*S3)

sin_beta2 = sqrt(1-cos_beta2**2)
beta2 = atan2(sin_beta2, cos_beta2)

theta2 = (pi/2 - beta1 - beta2).evalf()             

# 4. calculate theta3:
cos_beta3 = (s[a2]**2 + S4**2 - S3**2)/(2*s[a2]*S4)
sin_beta3 = sqrt(1 - cos_beta3**2)
beta3 = atan2(sin_beta3, cos_beta3)
beta4 = atan2(-s[a3], s[d4])

theta3 = (pi/2 - beta4 - beta3).evalf()

# 5. calculate theta4, 5, 6:
R3_6_num = R3_6.evalf(subs={q1: theta1, q2: theta2, q3: theta3, roll: ee_roll, pitch: ee_pitch, yaw: ee_yaw})

In [152]:
P_WC, theta1, theta2, theta3

(Matrix([
 [ 1.70754640499363],
 [0.939620269912033],
 [ 1.23497442497361]]),
 0.503054396122224,
 0.232694928426853,
 0.240249106827666)

In [150]:
s[a3], s[d4]

(-0.054, 1.5)

In [153]:
theta1-np.pi

-2.63853825746757

In [155]:
R3_6_num = R3_6.evalf(subs={q1: theta1, q2: theta2, q3: theta3, roll: ee_roll, pitch: ee_pitch, yaw: ee_yaw})

r13 = R3_6_num[0,2]
r33 = R3_6_num[2,2]

r21 = R3_6_num[1,0]
r22 = R3_6_num[1,1]

r23 = R3_6_num[1,2]

theta4 = atan2(r33, -r13).evalf()
theta6 = atan2(-r22, r21).evalf()

theta5 = atan2(sqrt(1 - r23**2), r23).evalf()

In [156]:
theta4, theta5, theta6.evalf()

(0.626731467903988, 0.423253864620880, 1.64935246812825)

In [157]:
theta4

0.626731467903988

In [158]:
R_z = Matrix([[  cos(pi), -sin(pi), 0, 0],
               [ sin(pi),  cos(pi), 0, 0],
               [       0,        0, 1, 0],
               [       0,        0, 0, 1]])

R_y = Matrix([[  cos(-pi/2), 0, sin(-pi/2), 0],
               [          0, 1,          0, 0],
               [-sin(-pi/2), 0, cos(-pi/2), 0],
               [          0, 0,          0, 1]])
R_corr = simplify(R_z * R_y)

In [160]:
(T0_G * R_corr).evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6})

Matrix([
[ 0.462223085829612,  0.882442140183783, 0.0874396257655269, 1.8476],
[  0.52930603989428, -0.195441866244314, -0.825613464673617,    1.1],
[-0.711466749087816,  0.427899925387706, -0.557419697172257, 1.0194],
[                 0,                  0,                  0,    1.0]])